In [1]:
import shap
import pandas as pd
from keras.utils import to_categorical
import pickle
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense, GRU, Dropout, Embedding
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [2]:
trained_model = open('../results/trained_net_rischio2_istat.p',"rb")
model = pickle.load(trained_model)
title = "../results/test_set_rischio2_istat_strat.p"
test = open(title,"rb")
test_set = pickle.load(test)
title = "../results/test_label_rischio2_istat_strat.p"
test_l = open(title,"rb")
test_label = pickle.load(test_l)
datas = open("../../datasets/Prato_Pistoia_trajectories_ids.p","rb")
dataset = pickle.load(datas)
coors = pd.read_csv('../../datasets/Prato_Pistoia_trajectories_df')
title = "../../shap/values_shap_lstm.p"
test_l = open(title,"rb")
shap_values = pickle.load(test_l)
title = "../results/explainer_1.p"
test_l = open(title,"rb")
expl_1 = pickle.load(test_l)
title = "../results/explainer_0.p"
test_l = open(title,"rb")
expl_0 = pickle.load(test_l)
title = "../results/shap_values_lstm_risultati.p"
test_l = open(title,"rb")
shaps = pickle.load(test_l)
freq = pd.read_csv('../../datasets/frequencies_entropy_istat.csv')
title = "../results/users_index"
test_l = open(title,"rb")
users = pickle.load(test_l)

Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.


In [3]:
#mapping ids to geo coordinates
maps = dict()
for index, row in coors.iterrows():
    maps[int(row['location'])]=(row['lat'], row['lon'])
maps_test = list()
for t in test_set:
    temp = list()
    for e in t:
        #this is padding 
        if e == 0:
            temp.append(e)
        #real data    
        else:
            temp.append(prova[e])
    maps_test.append(temp) 
result = list()
for n in maps_test:
    result.append(list(map(str, n)))

In [5]:
#remove 0 shap values and sort 
points = dict()
for r in shaps:
    sorted(shaps[r], key = lambda x : x[0] )
    shaps[r].sort(reverse=True)
    points[r] = list()
    for i in range(0, len(shaps[r])):
        points[r].append((i, shaps[r][i][0], shaps[r][i][1]))

In [8]:
import math

#for each user, for the top 6 locations, look at the frequency of visits for that location

risultato = dict()
for i in range(0, len(points.keys())):
    flag = False
    risultato[i] = list()
    f = freq.loc[freq['uid'] == users[i]]
    for e in range(0,len(points[i])):
        p = eval(points[i][e][2])
        count = 0
        if p != 0 and count<6:  
            for k in range(2, 20):
                if math.isclose(f['lat_'+str(k)], p[0]) & math.isclose(f['lng_'+str(k)], p[1]):
                    risultato[i].append(str(k))
                    risultato[i].append(f['entropy_'+str(k)].iloc[0])
                    risultato[i].append(f[str(k)].iloc[0])  
            count += 1

In [9]:
#predict the class for each element of the test set
import numpy as np
predictions = list()
for i in range(0, len(test_set)):
    p = np.array([test_set[i]])
    r = model.predict(p)[0]
    if r[0] > r[1]:
        predictions.append(0)
    else:
        predictions.append(1)
    

In [10]:
#for each prediction, take the index 
indexes_0 = list()
indexes_1 = list()
count = 0
for p in predictions:
    if p == 0:
        indexes_0.append(count)
    else:
        indexes_1.append(count)
    count += 1    
        

In [312]:
locs = list()
for i in indexes_0:
    if len(risultato[i]) != 0:
        try:
            locs.append(risultato[i][0])
        except:
            continue  

In [323]:
for i in indexes_1:
    if len(risultato[i]) != 0:
        try:
            locs.append(risultato[i][0])
        except:
            continue

In [478]:
#total number of visits for the test set
count = 0
for i in result:
    for e in i:
        if e != '0':
            count += 1
count            

54785

In [505]:
types_0 = list()
types_0_tot = list()
types_1 = list()
types_1_tot = list()
for i in indexes_0:
    if len(risultato[i]) != 0:
        try:
            types_0.append(risultato[i][2])
            count = 0
            for r in result[i]:
                r = eval(r)
                if r != 0:
                    count +=1 
            types_0_tot.append(count)        
        except:
            continue
for i in indexes_1:
    if len(risultato[i]) != 0:
        try:
            types_1.append(risultato[i][2])   
            count = 0
            for r in result[i]:
                r = eval(r)
                if r != 0:
                    count +=1 
            types_1_tot.append(count)             
        except:
            continue

In [506]:
ratio0 = list()
for t in range(0, len(types_0)):
    ratio0.append(types_0[t]/float(types_0_tot[t]))
ratio1 = list()
for t in range(0, len(types_1)):
    ratio1.append(types_1[t]/float(types_1_tot[t]))    

In [507]:
print(statistics.mean(ratio0))
print(statistics.stdev(ratio0))

0.605309797877666
0.2735967931182144


In [508]:
print(statistics.mean(ratio1))
print(statistics.stdev(ratio1))

0.35025536918649647
0.2617786797873071


In [465]:
counts = list()
for r in risultato:
    p = risultato[r][0]
    for t in result:
        c = 0
        for i in t:
            i = eval(i)
            if i != 0:
                if i[0] == p[0] and i[1] == p[1]:
                    c += 1
        counts.append(c)            

KeyboardInterrupt: 

In [469]:
lcs = dict()
for l in locs:
    lcs[l] = 0
    for r in result:
        lcs[l] += r.count(l)

In [485]:
types_0 = list()
types_0_tot = list()
types_1 = list()
types_1_tot = list()
for i in indexes_0:
    if len(risultato[i]) != 0:
        try:
            p = str(risultato[i][2])
            types_0.append(lcs[p]/float(54785) ) 
        except:
            continue
for i in indexes_1:
    if len(risultato[i]) != 0:
        try:
            p = str(risultato[i][2])
            types_1.append(lcs[p]/float(54785))              
        except:
            continue

In [486]:
print(statistics.mean(types_0))
print(statistics.stdev(types_0))

0.004395653636516095
0.005198279446571374


In [487]:
print(statistics.mean(types_1))
print(statistics.stdev(types_1))

0.002010998548220432
0.002739346557720432
